In [ ]:
!pip install pycountry

In [32]:
import pandas as pd
import numpy as np

In [ ]:
# Datos desde el local
data_matches = pd.read_csv("files/WorldCupMatches.csv")
data_players = pd.read_csv("files/WorldCupPlayers.csv")
data_wc = pd.read_csv("files/WorldCups.csv")

In [175]:
# Datos desde drive
data_matches = pd.read_csv("/content/drive/MyDrive/Carlos/Files/WorldCupMatches.csv")
data_fifa_23 = pd.read_csv('/content/drive/MyDrive/Carlos/Files/FIFA23_official_data.csv')

In [176]:
# Elimino Columnas a ojo
data_fifa_23.drop(['ID', 'Photo', 'Flag', 'Club', 'Club Logo', 'Release Clause', 'Kit Number', 'Contract Valid Until',
                   'Real Face', 'Body Type', 'Work Rate', 'Position', 'Loaned From', 'Name', 'Preferred Foot', 'Wage'], axis=1, inplace=True)

In [199]:
data_fifa_grouped = data_fifa_23.groupby(['Nationality']).mean()

In [200]:
data_fifa_grouped = data_fifa_grouped.reset_index(level=0)

## Matches DB

In [ ]:
# check data
data_matches.head()

In [ ]:
diccionario_

In [208]:
data_matches[data_matches['Away Team Name'] == 'Qatar'] 

,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,Home Team Initials,Away Team Initials


In [ ]:
# some of the 2014 matches have multiple entries which we must remove
data_matches[data_matches["MatchID"].isin(data_matches["MatchID"][data_matches["MatchID"].duplicated()])]

In [184]:
# Remove AR entry for 2014 final since it will not be removed by the func
data_matches.drop(851, inplace=True)

In [185]:
# Remove Repeated entries
data_matches.drop_duplicates(inplace=True)

In [186]:
# Drop some columns by eye
data_matches.drop(['Referee', 'Assistant 1', 'Assistant 2', 'RoundID', 'MatchID', 'Attendance', 'Stadium', 'City', 'Datetime'], axis=1, inplace=True)

In [187]:
data_matches_model = data_matches.copy()

In [188]:
# filter by wc attendants
data_matches_model = data_matches_model[(data_matches_model['Home Team Name'].isin(countries_in_wc)) & (data_matches_model['Away Team Name'].isin(countries_in_wc))]

In [191]:
data_matches_model[data_matches_model['Away Team Name'] == 'Iran'] 

,Year,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Half-time Home Goals,Half-time Away Goals,Home Team Initials,Away Team Initials
558,1998,Group F,Germany,2,0,Iran,,0,0,GER,IRN
650,2006,Group D,Mexico,3,1,Iran,,1,1,MEX,IRN


In [192]:
# Functions that will help cleaning the dataset
def get_winner(df, cat=False):
    """
    Get the winner from the dataset based on goals
    """
    if df['Home Team Goals'] > df['Away Team Goals']:
        return df['Home Team Initials'] if not cat else 1
    return df['Away Team Initials'] if not cat else 0

def get_country_name(df):
  """
  Get the country names
  """
  try:
    return pycountry.countries.get(name=df['Nationality']).alpha_3
  except:
    return None

def get_loser(df):
    """
    Get the loser from the dataset based on goals
    """
    if df['Home Team Goals'] < df['Away Team Goals']:
        return df['Home Team Initials']
    return df['Away Team Initials']


def clean_stage(df):
    """
    Remove the groups x and replace it just with Groups
    """
    if df['Stage'].__contains__('Group'):
        return 'Group'
    return df['Stage']

def loser(rows):
    c = 0
    for row in rows:
        print(row)
        if row["Loser Team"] == row['Winner Team']:
            c += 1

    return c 

In [193]:
data_matches_model['Winner Team'] = data_matches_model.apply(lambda row: get_winner(row), axis=1)
data_matches_model['Loser Team'] = data_matches_model.apply(lambda row: get_loser(row), axis=1)
data_matches_model['Stage'] = data_matches_model.apply(lambda row: clean_stage(row), axis=1)

In [194]:
loses_count = data_matches_model.groupby(["Loser Team", "Stage"], as_index=False)["Year"].count().rename(columns={'Year': 'Loses', "Loser Team": "Country"})

In [195]:
winner_count = data_matches_model.groupby(["Winner Team", "Stage"], as_index=False)["Year"].count().rename(columns={"Winner Team": "Country", 'Year': 'Wins'})

In [196]:
agg_data = data_matches_model.groupby(["Winner Team", "Stage"], as_index=False)['Home Team Goals', "Away Team Goals", "Half-time Home Goals", "Half-time Away Goals"].mean().rename(columns={"Winner Team": "Country"})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [ ]:
import functools as ft
data_matches_final = ft.reduce(lambda left, right: pd.merge(left, right, on=['Country', "Stage"]), [loses_count, winner_count, agg_data])
data_matches_final

In [201]:
data_fifa_grouped['Country Code'] = data_fifa_grouped.apply(lambda row: get_country_name(row), axis=1)

In [202]:
# Fix the missing countries
data_fifa_grouped[data_fifa_grouped['Country Code'].isnull()]
missing_countries = {'Wales': 'WAL', 'England': 'ENG', 'Germany': 'GER', 'Costa Rica': 'CRC'}

In [203]:
data_merged = pd.merge(data_matches_final, data_fifa_grouped, left_on='Country', right_on='Country Code', how='left')

In [ ]:
data_merged

## Model

In [205]:
data_matches_model.drop(["Year", "Home Team Name", "Win conditions", "Away Team Name"], inplace=True, axis=1)

In [206]:
data_matches_model['Winner Team'] = data_matches_model.apply(lambda row: get_winner(row, cat=True), axis=1)
data_matches_model['Stage'] = data_matches_model.apply(lambda row: clean_stage(row), axis=1)

In [207]:
pd.get_dummies(data_matches_model, columns=["Stage"],prefix=["Stage_"])

,Home Team Goals,Away Team Goals,Half-time Home Goals,Half-time Away Goals,Home Team Initials,Away Team Initials,Winner Team,Loser Team,Stage__Final,Stage__First round,Stage__Group,Stage__Match for third place,Stage__Play-off for third place,Stage__Preliminary round,Stage__Quarter-finals,Stage__Round of 16,Stage__Semi-finals,Stage__Third place
0,4,1,3,0,FRA,MEX,1,MEX,0,0,1,0,0,0,0,0,0,0
4,1,0,0,0,ARG,FRA,1,FRA,0,0,1,0,0,0,0,0,0,0
10,6,3,3,1,ARG,MEX,1,MEX,0,0,1,0,0,0,0,0,0,0
17,4,2,1,2,URU,ARG,1,ARG,1,0,0,0,0,0,0,0,0,0
20,3,2,2,1,SUI,NED,1,NED,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
829,0,0,0,0,NED,ARG,0,ARG,0,0,0,0,0,0,0,0,1,0
830,0,0,0,0,NED,CRC,0,CRC,0,0,0,0,0,0,1,0,0,0
831,1,0,1,0,ARG,BEL,1,BEL,0,0,0,0,0,0,1,0,0,0
832,2,1,0,0,NED,MEX,1,MEX,0,0,0,0,0,0,0,1,0,0
